In [149]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, Polygon

In [133]:
df_2015_q2 = pd.read_csv('../Data/TripData/trip_data_2015Q2.csv')
stations_centract_df = pd.read_csv('../Data/stations_census_tracts.csv')

In [134]:
df_2015_q2.columns

Index(['Trip id', 'Starttime', 'Stoptime', 'Bikeid', 'Tripduration',
       'From station id', 'From station name', 'To station id',
       'To station name', 'Usertype'],
      dtype='object')

In [135]:
df_2015_q2 = df_2015_q2.rename(columns={'Trip id': 'Tripid', 'From station id': 'From_station_id',
                                       'From station name': 'From_station_name', 'To station id': 
                                       'To_station_id', 'To station name': 'To_station_name'})
df_2015_q2.head(1)

,Tripid,Starttime,Stoptime,Bikeid,Tripduration,From_station_id,From_station_name,To_station_id,To_station_name,Usertype
0,10676972,5/31/2015 6:54,5/31/2015 10:17,70166,12190,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer


In [136]:
stations_centract_df.columns

Index(['Station #', 'Station_Name', 'NumRacks', 'Latitude', 'Longitude',
       'coordinates', 'tractce10'],
      dtype='object')

In [137]:
stations_centract_df = stations_centract_df.rename(columns={'Station #': 'station_id'})
stations_centract_df.head(2)

,station_id,Station_Name,NumRacks,Latitude,Longitude,coordinates,tractce10
0,1000,Liberty Ave & Stanwix St,16,40.441326,-80.004679,POINT (-80.00467900000001 40.441326),20100
1,1001,Forbes Ave & Market Square,19,40.440877,-80.003080,POINT (-80.00308000000001 40.440877),20100


In [146]:
type(stations_centract_df.loc[0]['tractce10'])

numpy.int64

In [138]:
df_2015_q2.insert(7, "from_census_tract", 0)

In [139]:
df_2015_q2.insert(10, "to_census_tract", 0)
df_2015_q2.head(1)

,Tripid,Starttime,Stoptime,Bikeid,Tripduration,From_station_id,From_station_name,from_census_tract,To_station_id,To_station_name,to_census_tract,Usertype
0,10676972,5/31/2015 6:54,5/31/2015 10:17,70166,12190,1018,Butler St & Stanton Ave,0,1001,Forbes Ave & Market Square,0,Customer


In [140]:
centract_dict = dict(zip(stations_centract_df.station_id, stations_centract_df.tractce10))
centract_dict

{1000: 20100,
 1001: 20100,
 1002: 20100,
 1003: 20100,
 1004: 20100,
 1005: 20100,
 1006: 20100,
 1007: 10300,
 1008: 20100,
 1009: 20300,
 1010: 20100,
 1011: 20100,
 1012: 981200,
 1013: 563200,
 1014: 562700,
 1015: 562700,
 1016: 20300,
 1017: 20300,
 1018: 981000,
 1019: 90100,
 1020: 90200,
 1021: 90300,
 1022: 80900,
 1023: 80600,
 1024: 111500,
 1025: 111400,
 1026: 111500,
 1027: 70800,
 1028: 70800,
 1029: 70800,
 1031: 70500,
 1032: 70500,
 1033: 70300,
 1034: 70900,
 1035: 982200,
 1036: 982200,
 1037: 140100,
 1038: 40600,
 1039: 40600,
 1040: 982200,
 1041: 40500,
 1042: 50100,
 1043: 80400,
 1044: 40500,
 1045: 160900,
 1046: 160900,
 1047: 160900,
 1048: 170200,
 1049: 170200,
 1052: 10300,
 1053: 120800,
 1054: 20100,
 1056: 20100,
 1059: 20300,
 1060: 20300,
 1061: 60300,
 1062: 80900,
 1063: 101700,
 1064: 111500,
 1066: 130300,
 1068: 140800,
 1069: 141300,
 1070: 141300,
 1072: 980700,
 1074: 170200,
 1084: 160900,
 1088: 40900,
 1091: 40500,
 1093: 40500,
 1094: 

In [141]:
centract_dict.get(49951)
#return the census tract
#parameter is the station id

60300

In [142]:
for i in range(len(df_2015_q2)):
    from_station = df_2015_q2.loc[i]['From_station_id']
    
    if from_station == 1050:
        df_2015_q2.at[i, 'from_census_tract'] = 0
    else:
        df_2015_q2.at[i, 'from_census_tract'] = centract_dict.get(from_station)
    
    to_station = df_2015_q2.loc[i]['To_station_id']
    
    if to_station == 1050:
        df_2015_q2.at[i, 'to_census_tract'] = 0
    else:
        df_2015_q2.at[i, 'to_census_tract'] = centract_dict.get(to_station)

df_2015_q2.to_csv('../Data/TripData/trip_data_2015Q2_ct.csv')

In [147]:
df_2015_q2 = pd.read_csv('../Data/TripData/trip_data_2015Q2_ct.csv')
for i in range(len(df_2015_q2)):
    if pd.isna(df_2015_q2.loc[i]['to_census_tract']):
        df_2015_q2.at[i, 'to_census_tract'] = 0
        
    if pd.isna(df_2015_q2.loc[i]['from_census_tract']):
        df_2015_q2.at[i, 'from_census_tract'] = 0    

In [148]:
df_2015_q2.astype({'from_census_tract': 'int64', 'to_census_tract': 'int64'})

,Unnamed: 0,Tripid,Starttime,Stoptime,Bikeid,Tripduration,From_station_id,From_station_name,from_census_tract,To_station_id,To_station_name,to_census_tract,Usertype
0,0,10676972,5/31/2015 6:54,5/31/2015 10:17,70166,12190,1018,Butler St & Stanton Ave,981000,1001,Forbes Ave & Market Square,20100,Customer
1,1,10677086,5/31/2015 6:57,5/31/2015 9:24,70368,8842,1018,Butler St & Stanton Ave,981000,1018,Butler St & Stanton Ave,981000,Subscriber
2,2,10677131,5/31/2015 6:58,5/31/2015 10:09,70354,11448,1019,42nd St & Butler St,90100,1019,42nd St & Butler St,90100,Subscriber
3,3,10677525,5/31/2015 7:08,5/31/2015 7:15,70232,452,1001,Forbes Ave & Market Square,20100,1001,Forbes Ave & Market Square,20100,Subscriber
4,4,10677886,5/31/2015 7:15,5/31/2015 9:27,70468,7866,1018,Butler St & Stanton Ave,981000,1018,Butler St & Stanton Ave,981000,Subscriber
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9193,9193,11714255,6/30/2015 22:43,6/30/2015 22:45,70175,103,1012,North Shore Trail & Fort Duquesne Bridge,981200,1012,North Shore Trail & Fort Duquesne Bridge,981200,Customer
9194,9194,11714259,6/30/2015 22:44,6/30/2015 22:51,70038,417,1012,North Shore Trail & Fort Duquesne Bridge,981200,1001,Forbes Ave & Market Square,20100,Customer
9195,9195,11714328,6/30/2015 23:16,6/30/2015 23:50,70363,2043,1014,Ridge Ave & Brighton Rd (CCAC),562700,1015,Federal St & E North Ave,562700,Customer
9196,9196,11714401,6/30/2015 23:35,6/30/2015 23:53,70477,1061,1036,Schenley Dr at Schenley Plaza (Carnegie Librar...,982200,1039,Atwood St & Bates St,40600,Subscriber


In [ ]:
inflow_dict = {}
outflow_dict = {}

